# Import libraries

In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

# Load prepared dataframe

In [49]:
df = pd.read_csv('../data/df_prepped.csv')
pd.set_option('display.max_columns', None)

print('df.shape:', df.shape)
df.head()

df.shape: (32359, 50)


,Year,Countries,Sand_1,Sand_2,Sand_3,Sand_4,Sand_5,Sand_6,Sand_7,Clay_1,Clay_2,Clay_3,Clay_4,Clay_5,Clay_6,Clay_7,OC_1,OC_2,OC_3,OC_4,OC_5,OC_6,OC_7,PAW_1,PAW_2,PAW_3,PAW_4,PAW_5,PAW_6,PAW_7,Y_maize_major,Farm,Sow_Maize_month_int,Harvest_Maize_month_int,sow_to_harvest_months,maize_lag-1,pcp_mean_lag-1,tmax_mean_lag-1,tmin_mean_lag-1,spi_mean_lag-1,maize_lag-2,pcp_mean_lag-2,tmax_mean_lag-2,tmin_mean_lag-2,spi_mean_lag-2,maize_lag-3,pcp_mean_lag-3,tmax_mean_lag-3,tmin_mean_lag-3,spi_mean_lag-3
0,2007,Angola,50,51,51,48,45,46,46,37,35,36,39,42,42,42,0.52,0.23,0.17,0.09,0.04,0.02,0.02,0.15,0.15,0.14,0.13,0.10,0.07,0.07,0.615357,104_Angola,9,4,7,0.554392,97.103755,301.939623,292.214020,0.093447,0.721607,129.051864,301.518536,292.496579,1.644698,0.620005,109.983325,301.786056,292.204097,0.514275
1,2007,Angola,62,64,63,59,58,59,59,27,25,26,29,31,30,30,0.11,0.05,0.07,0.04,0.02,0.02,0.01,0.11,0.10,0.10,0.09,0.07,0.07,0.03,0.257656,99_Angola,9,4,7,0.117051,59.292237,301.882929,288.092753,0.182926,0.300217,47.697564,303.988747,288.916992,0.909295,0.212699,41.130026,303.298082,288.642853,0.588172
2,2007,Angola,69,71,70,67,65,65,66,19,16,18,21,24,24,23,0.09,0.06,0.07,0.04,0.02,0.02,0.02,0.10,0.10,0.10,0.09,0.07,0.07,0.07,4.286831,108_Angola,9,4,7,3.093239,58.196545,302.891420,289.377311,0.991663,4.044452,42.130629,305.494178,290.535403,0.952237,2.295351,35.049776,304.824778,290.284886,0.371446
3,2007,Angola,60,63,61,57,53,53,53,29,26,28,32,35,36,36,0.46,0.16,0.14,0.08,0.05,0.04,0.03,0.12,0.13,0.12,0.12,0.11,0.10,0.09,0.700384,102_Angola,9,4,7,0.677797,149.210195,298.973795,287.311403,0.206751,0.907431,159.454723,299.404975,287.724299,1.374616,0.783018,174.088260,298.908208,287.362407,0.643207
4,2007,Angola,67,69,68,63,61,61,61,22,19,21,25,28,28,29,0.15,0.09,0.09,0.05,0.02,0.01,0.01,0.11,0.11,0.11,0.11,0.08,0.04,0.04,0.553450,43_Angola,9,4,7,0.412071,74.556629,304.006860,290.606725,-0.075621,0.675967,66.698670,304.644632,290.635254,1.144088,0.605584,67.404588,303.930955,290.564185,0.553079


In [50]:
df.Countries.nunique()

30

In [51]:
df.Farm.nunique()

3887

In [52]:
df.isna().sum()

Year                       0
Countries                  0
Sand_1                     0
Sand_2                     0
Sand_3                     0
Sand_4                     0
Sand_5                     0
Sand_6                     0
Sand_7                     0
Clay_1                     0
Clay_2                     0
Clay_3                     0
Clay_4                     0
Clay_5                     0
Clay_6                     0
Clay_7                     0
OC_1                       0
OC_2                       0
OC_3                       0
OC_4                       0
OC_5                       0
OC_6                       0
OC_7                       0
PAW_1                      0
PAW_2                      0
PAW_3                      0
PAW_4                      0
PAW_5                      0
PAW_6                      0
PAW_7                      0
Y_maize_major              0
Farm                       0
Sow_Maize_month_int        0
Harvest_Maize_month_int    0
sow_to_harvest

# Train-test split

In [53]:
# Dropping countries and farms because one-hot-encoding would make 4000+ rows
# We tested that and found that it did not improve performance but increased runtime
df_label = df.loc[:,['Countries','Farm']]
df = df.drop(['Countries','Farm'], axis=1)

In [54]:
# Separate a test set, the year 2016
test = df[df.Year == 2016]
df_train = df[df.Year < 2016]

In [55]:
print('The training set has years: ', list(df_train.Year.unique()))
print('The test set has years: ', list(test.Year.unique()))

The training set has years:  [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
The test set has years:  [2016]


# Baseline selection using CV and years 2007-2015

In [56]:
# Cross-validation set-up
# https://stackoverflow.com/questions/58069691/how-to-create-a-train-test-split-of-time-series-data-by-year

year_list = df_train['Year'].unique().tolist()
splits = {'train': [], 'val': []}

for idx, yr in enumerate(year_list[:-1]):
    if yr < 2010:
        # To get only the last 5 splits
        continue
    train_yr = year_list[:idx+1]
    test_yr = [year_list[idx+1]]
    print('TRAIN: ', train_yr, 'VAL: ',test_yr)
    
    splits['train'].append(df_train.loc[df_train.Year.isin(train_yr), :])
    splits['val'].append(df_train.loc[df_train.Year.isin(test_yr), :])

TRAIN:  [2007, 2008, 2009, 2010] VAL:  [2011]
TRAIN:  [2007, 2008, 2009, 2010, 2011] VAL:  [2012]
TRAIN:  [2007, 2008, 2009, 2010, 2011, 2012] VAL:  [2013]
TRAIN:  [2007, 2008, 2009, 2010, 2011, 2012, 2013] VAL:  [2014]
TRAIN:  [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014] VAL:  [2015]


In [57]:
SEED = 1

In [58]:
regressors = {
    "KNN": KNeighborsRegressor(), # 1 cannot extrapolate
    "Random Forest": RandomForestRegressor(random_state=SEED), # 2 cannot extrapolate
    "Gradient Boosting": AdaBoostRegressor(random_state=SEED), # 3 cannot extrapolate
    "Linear Regression": LinearRegression(), # 4 can extrapolate
    "LGBM": lgb.sklearn.LGBMRegressor(random_state=SEED) # 5 can extrapolate
    
}

In [59]:
# This will take about 15 minutes

reg_names = []
cv_MAE = []
cv_RMSE = []
cv_folds = []
train_years = []
val_year = []

i = 1
for train, val in zip(splits['train'], splits['val']):

    print('Fold: ', i)

    # Shuffle
    train = train.sample(frac=1, random_state=SEED)
    val = val.sample(frac=1, random_state=SEED)
    
    # X and y
    X_train = train.drop(columns=['Y_maize_major','Year'], axis=1)
    y_train = train['Y_maize_major']
    X_val = val.drop(columns=['Y_maize_major','Year'], axis=1)
    y_val = val['Y_maize_major']

    # Scale to [0,1] range
    sc = MinMaxScaler()
    X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
    X_val = pd.DataFrame(sc.transform(X_val), columns=X_val.columns)

    # Fit and predict
    for reg_name, reg in regressors.items():
        reg_names.append(reg_name)
        reg.fit(X_train, y_train)
        y_val_pred = reg.predict(X_val)
        rmse = mean_squared_error(y_val, y_val_pred, squared=False)
        mae = mean_absolute_error(y_val, y_val_pred)
        cv_MAE.append(mae)
        cv_RMSE.append(rmse)
        cv_folds.append(i)
        train_years.append(list(train.Year.unique()))
        val_year.append(list(val.Year.unique()))

        print(reg_name)
        print('RMSE:', rmse)
        print('MAE:', mae)

    i += 1
    print('###')
    print()

Fold:  1


KNN
RMSE: 0.44075591508811046
MAE: 0.2936247010209556
Random Forest
RMSE: 0.35625256905189046
MAE: 0.23214587564870676
Gradient Boosting
RMSE: 0.4875346227174587
MAE: 0.3971658820206708
Linear Regression
RMSE: 0.3802067681736373
MAE: 0.2581393745105981
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002314 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5314
[LightGBM] [Info] Number of data points in the train set: 15512, number of used features: 46
[LightGBM] [Info] Start training from score 1.636864
LGBM
RMSE: 0.40319203965072503
MAE: 0.2548949593684437
###

Fold:  2
KNN
RMSE: 0.45573247287781565
MAE: 0.29140229382129695
Random Forest
RMSE: 0.3814279850259372
MAE: 0.22931856221496164
Gradient Boosting
RMSE: 0.5103637162242142
MAE: 0.3757062501105622
Linear Regression
RMSE: 0.4125156152014488
MAE: 0.252363280898665
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing wa

In [60]:
df_results = pd.DataFrame()
df_results['fold'] = cv_folds
df_results['regressor'] = reg_names
df_results['RMSE'] = cv_RMSE
df_results['MAE'] = cv_MAE
df_results['train_years'] = train_years
df_results['val_year'] = val_year
df_results

,fold,regressor,RMSE,MAE,train_years,val_year
0,1,KNN,0.440756,0.293625,"[2008, 2007, 2010, 2009]",[2011]
1,1,Random Forest,0.356253,0.232146,"[2008, 2007, 2010, 2009]",[2011]
2,1,Gradient Boosting,0.487535,0.397166,"[2008, 2007, 2010, 2009]",[2011]
3,1,Linear Regression,0.380207,0.258139,"[2008, 2007, 2010, 2009]",[2011]
4,1,LGBM,0.403192,0.254895,"[2008, 2007, 2010, 2009]",[2011]
5,2,KNN,0.455732,0.291402,"[2009, 2010, 2007, 2008, 2011]",[2012]
6,2,Random Forest,0.381428,0.229319,"[2009, 2010, 2007, 2008, 2011]",[2012]
7,2,Gradient Boosting,0.510364,0.375706,"[2009, 2010, 2007, 2008, 2011]",[2012]
8,2,Linear Regression,0.412516,0.252363,"[2009, 2010, 2007, 2008, 2011]",[2012]
9,2,LGBM,0.423686,0.252619,"[2009, 2010, 2007, 2008, 2011]",[2012]


In [61]:
df_results.to_csv('../experiment_results/baseline_selection_results_Kea.csv',index=False)

In [69]:
df_results_agg = df_results.groupby('regressor').agg({'RMSE': ['mean'], 'MAE': [ 'mean']})
df_results_agg

,RMSE,MAE
,mean,mean
regressor,,
Gradient Boosting,0.499022,0.372772
KNN,0.469980,0.302485
LGBM,0.425715,0.250691
Linear Regression,0.391492,0.239065
Random Forest,0.410952,0.245647


Linear regression is has lowest RMSE and MAE.

In [70]:
## How to find from results mean RMSE for Random Forest 
mean_RMSE_best = df_results[df_results['regressor'] == 'Linear Regression'].agg({'RMSE': ['mean']}).values[0][0]
mean_MAE_best = df_results[df_results['regressor'] == 'Linear Regression'].agg({'MAE': ['mean']}).values[0][0]


In [71]:
print("As we can see, Linear Regression has the best performance in both RMSE and MAE. \n\nMean RMSE is:\t {:.4f}".format(mean_RMSE_best))
print("Mean MAE is:\t {:.4f}".format(mean_MAE_best))


As we can see, Linear Regression has the best performance in both RMSE and MAE. 

Mean RMSE is:	 0.3915
Mean MAE is:	 0.2391


## Refit baseline on train and predict on test set

In [73]:
print('The training set has years: ', list(df_train.Year.unique()))
print('The test set has years: ', list(test.Year.unique()))

The training set has years:  [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
The test set has years:  [2016]


In [74]:
# Shuffle
train = df_train.sample(frac=1, random_state=SEED)
test = test.sample(frac=1, random_state=SEED)

# X and y
X_train = train.drop(columns=['Y_maize_major','Year'], axis=1)
y_train = train['Y_maize_major']
X_test = test.drop(columns=['Y_maize_major','Year'], axis=1)
y_test = test['Y_maize_major']

# Scale to [0,1] range
sc = MinMaxScaler()
X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(sc.transform(X_test), columns=X_test.columns)

In [75]:
# Fit on train and predict on test
reg = regressors['Linear Regression']
reg.fit(X_train, y_train)
y_test_pred = reg.predict(X_test)

In [76]:
# Calculate and print metrics
rmse = mean_squared_error(y_test, y_test_pred, squared=False)
mae = mean_absolute_error(y_test, y_test_pred)
print('Performance on test set (year 2016) (baseline results)')
print('RMSE:',round(rmse,4))
print('MAE:',round(mae,4))

Performance on test set (year 2016) (baseline results)
RMSE: 0.3288
MAE: 0.2161
